# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [46]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
# Import API key
from api_keys import geoapify_key
print(geoapify_key)

a7b9c987e8464e43aa696aa21cfcff2e


In [75]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,16.83,67,64,216,GS,1722998889
1,1,kirakira,-10.4544,161.9205,82.85,74,52,108,SB,1722998891
2,2,labytnangi,66.6572,66.4183,53.15,55,2,68,RU,1722998893
3,3,dehdasht,30.7949,50.5646,87.35,19,6,95,IR,1722998895
4,4,albany,42.6001,-73.9662,64.62,94,100,14,US,1722998897


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [76]:
%%capture --no-display
humidity = city_data_df["Humidity"].astype(int)

# Configure the map plot
map_Hum = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75
)

# Display the map
map_Hum


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [102]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = pd.read_csv("output_data/cities.csv")
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 4.5)]
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 50)]
city_data_df = city_data_df.loc[(city_data_df["Humidity"] > 91)]
# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,11,ilulissat,69.2167,-51.1000,44.62,93,62,0,GL,1722998344
32,32,nuuk,64.1835,-51.7216,47.01,93,100,0,GL,1722998978
76,76,longyearbyen,78.2186,15.6401,48.04,93,75,0,SJ,1722999076
166,166,ivdel',60.6911,60.4206,49.50,95,100,1,RU,1722999347


### Step 3: Create a new DataFrame called `hotel_df`.

In [103]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
city = city_data_df["City"]
country = city_data_df["Country"]
lat = city_data_df["Lat"]
lng = city_data_df["Lng"]
humidity = city_data_df["Humidity"]

hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]]
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API


# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity
11,ilulissat,GL,69.2167,-51.1000,93
32,nuuk,GL,64.1835,-51.7216,93
76,longyearbyen,SJ,78.2186,15.6401,93
166,ivdel',RU,60.6911,60.4206,95


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [104]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search


C:\Users\cxb\AppData\Local\Temp\ipykernel_28816\854976939.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]


ilulissat - nearest hotel: Best Western Plus Hotel Ilulissat
nuuk - nearest hotel: Nordbo
longyearbyen - nearest hotel: The Vault
ivdel' - nearest hotel: гостиница Ивдель


,City,Country,Lat,Lng,Humidity,Hotel Name
11,ilulissat,GL,69.2167,-51.1000,93,Best Western Plus Hotel Ilulissat
32,nuuk,GL,64.1835,-51.7216,93,Nordbo
76,longyearbyen,SJ,78.2186,15.6401,93,The Vault
166,ivdel',RU,60.6911,60.4206,95,гостиница Ивдель


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [106]:
%%capture --no-display

# Configure the map plot
map_hotel = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_hotel

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)